In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [3]:
img = load_img('data/rust/masked_1.jpg')  # this is a PIL image

FileNotFoundError: [Errno 2] No such file or directory: 'data/rust/masked_1.jpg'

In [5]:
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

In [6]:
x.shape

(1, 2048, 1152, 3)

In [8]:
# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='data/rust_generated', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [4]:
!conda install PIL

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - pil

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [13]:
img_shape = (150, 150)

In [14]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=img_shape + (3,)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [15]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [16]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=img_shape,  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=img_shape,
        batch_size=batch_size,
        class_mode='binary')

Found 1247 images belonging to 2 classes.
Found 313 images belonging to 2 classes.


In [17]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

Epoch 1/50
125/125 [==============================] - 107s 859ms/step - loss: 0.6806 - accuracy: 0.5788 - val_loss: 0.5838 - val_accuracy: 0.6310
Epoch 2/50
125/125 [==============================] - 98s 782ms/step - loss: 0.6417 - accuracy: 0.6471 - val_loss: 0.4569 - val_accuracy: 0.6932
Epoch 3/50
125/125 [==============================] - 93s 745ms/step - loss: 0.6241 - accuracy: 0.6612 - val_loss: 0.5087 - val_accuracy: 0.7265
Epoch 4/50
125/125 [==============================] - 98s 782ms/step - loss: 0.5871 - accuracy: 0.6877 - val_loss: 0.5876 - val_accuracy: 0.7407
Epoch 5/50
125/125 [==============================] - 92s 736ms/step - loss: 0.5755 - accuracy: 0.7204 - val_loss: 0.5349 - val_accuracy: 0.7112
Epoch 6/50
125/125 [==============================] - 96s 772ms/step - loss: 0.5319 - accuracy: 0.7447 - val_loss: 0.4523 - val_accuracy: 0.6893
Epoch 7/50
125/125 [==============================] - 104s 835ms/step - loss: 0.4899 - accuracy: 0.7829 - val_loss: 0.3047 - val_

In [18]:
model.save_weights('first_try.h5')  # always save your weights after training or during training